In [1]:
import functools

import numpy as np
import pennylane as qml
from scipy.linalg import expm

from graddft_qnn.dft_qnn import DFTQNN
from graddft_qnn.unitary_rep import O_h

# Defining...

## Unitary reps

In [2]:
unitary_reps = O_h._180_deg_rot()

In [3]:
unitary_reps

[array([[0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])]

## Ansatz

In [4]:
XII = functools.reduce(np.kron, [qml.X.compute_matrix(), np.eye(2), np.eye(2)])
IXI = functools.reduce(np.kron, [np.eye(2), qml.X.compute_matrix(), np.eye(2)])
IIX = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.X.compute_matrix()])

In [5]:
YII = functools.reduce(np.kron, [qml.Y.compute_matrix(), np.eye(2), np.eye(2)])
IYI = functools.reduce(np.kron, [np.eye(2), qml.Y.compute_matrix(), np.eye(2)])
IIY = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Y.compute_matrix()])

In [6]:
ZII = functools.reduce(np.kron, [qml.Z.compute_matrix(), np.eye(2), np.eye(2)])
IZI = functools.reduce(np.kron, [np.eye(2), qml.Z.compute_matrix(), np.eye(2)])
IIZ = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Z.compute_matrix()])

$$
ZZ(\phi) = \exp\left(-i \frac{\phi}{2} (Z \otimes Z)\right) =
\begin{bmatrix}
    e^{-i \phi / 2} & 0 & 0 & 0 \\
    0 & e^{i \phi / 2} & 0 & 0 \\
    0 & 0 & e^{i \phi / 2} & 0 \\
    0 & 0 & 0 & e^{-i \phi / 2}
\end{bmatrix}
$$

$ZZ(0) = I$

In [7]:
ZZZ_gen = (
    -1j
    * np.pi
    / 3
    * functools.reduce(
        np.kron,
        [qml.Z.compute_matrix(), qml.Z.compute_matrix(), qml.Z.compute_matrix()],
    )
)

expm(-1 theta X) exmp(-1 theta Z) -> measurement XZ
any output of the twirling is measurement, and generator

In [8]:
ZZZ = expm(ZZZ_gen)

In [9]:
def process(gate_matrix, u_reprs: list[np.array]):
    gen = DFTQNN.twirling(gate_matrix, unitary_reps=u_reprs)
    if isinstance(gen, np.ndarray):
        return qml.pauli_decompose(
            gen, check_hermitian=False, hide_identity=True, pauli=True
        )
    return None

# Run the twirling + generator

In [10]:
process(XII, unitary_reps), process(IXI, unitary_reps), process(IIX, unitary_reps)

((1+0j) * X(0), (1+0j) * X(1), (1+0j) * X(2))

In [11]:
process(YII, unitary_reps), process(IYI, unitary_reps), process(IIY, unitary_reps)

(None, None, (1+0j) * Y(2))

In [12]:
process(ZII, unitary_reps), process(IZI, unitary_reps), process(IIZ, unitary_reps)

(None, None, (1+0j) * Z(2))

# Design a circuit

In [13]:
expm(-1j * 2 * qml.X.compute_matrix())

array([[-0.41614684+0.j        ,  0.        -0.90929743j],
       [ 0.        -0.90929743j, -0.41614684+0.j        ]])

In [14]:
qml.RX.compute_matrix(4)

array([[-0.41614684+0.j        ,  0.        -0.90929743j],
       [ 0.        -0.90929743j, -0.41614684+0.j        ]])